# GenVeg Main Script
GenVeg is a generalized vegetation growth model that simulates growth due to
photosynthesis as well as processes that control vegetation distribution such
as mortality, senescence, dispersal, and colonization. The model utilizes a 
mixture of existing and new landlab components to setup the model, simulates
relevant vegetation community dynamics with the local environment, and analyzes
results of the simulation. 

## Main model control set up
### Import components

In [1]:
import numpy as np, pandas as pd
import os, tempfile
from pathlib import Path
from datetime import date
from matplotlib.pyplot import title, show, figure, plot, subplot, xlabel, ylabel
from landlab import RasterModelGrid, imshow_grid, load_params
from landlab.data_record import DataRecord
from landlab.components.genveg import VegParams, PlantGrowth
from landlab.io.netcdf import write_netcdf

### Define time controls

In [2]:
sim_start_date = date(2010,1,1)
tot_sim_len_y = 30
veg_time_step_d = 1
env_time_step_d = 1

### Define filenames for input files
##### *Note should be located in same folder as this .ipynb

In [3]:
VegInputFile = 'GenVeg_params_inputs_1col_TS.xlsx'

### Read in parameter and data input files
Identify model processes to be included so all required parameters are defined and calculated

In [4]:
inp_file=Path(VegInputFile).absolute()
#VegParams(fpath=inp_file,processes=['plantsize','dispersal','colonization','storage'])
VegParams(processes=['plantsize','dispersal','colonization','storage'])
vegparams=load_params('veg_params.yml')
print(vegparams)


{'Corn': {'colparams': {}, 'dispparams': {}, 'growthparams': {'glucose_requirement': [1.444, 1.513, 1.463], 'growing_season_end': [290], 'growing_season_start': [91], 'k_light_extinct': [0.02], 'light_half_sat': [9], 'p_max': [0.055], 'plant_part_allocation': [0.2, 0.3, 0.5], 'plant_part_min': [0.01, 0.1, 0.5], 'ptype': ['C3'], 'respiration_coefficient': [0.015, 0.015, 0.03], 'senescence_start': [228]}, 'mortparams': {}, 'plant_factors': {'annual_perennial': ['C4'], 'growth_form': [1], 'species': ['Corn']}, 'storparams': {'r_wint_die': [0.25], 'r_wint_stor': [0.25]}}, 'sizeparams': {'max_height_stem': [2.5], 'max_mass_stem': [72], 'max_n_stems': [3], 'max_plant_density': [1], 'total_cs_area_stems': [0.231]}}


### Set up model grid using landlab RasterModelGrid functions

- Create the model grid
- Initialize live vegetation biomass and plant functional type
- Initialize the air temperature driver across the grid

##### **Note variable names should be consistent with landlab standards

In [ ]:
#Create grid
pg = RasterModelGrid((100, 100), 2.0)
pg.axis_units = ('m', 'm')

#Initialize plant functional type as grass or bare
_ = pg.add_field('vegetation__plant_functional_type',np.random.choice([0,2],pg.number_of_cells), at='cell')

#Initialize biomass to random quantity between 0 and 200 g/m^2
_ = pg.add_field('vegetation__live_biomass',np.where(pg.at_cell['vegetation__plant_functional_type'] != 0,1,0)*
                np.random.uniform(low=0, high=200,size=pg.number_of_cells),at='cell', units='g')

#Initialize with a random mean air temperture 
_ = pg.add_field('air__mean_temp', np.random.uniform(low=12, high=14,size=pg.number_of_cells), at='cell', units='C' )

### Plot grid with plant distribution and initial biomass

Use RasterModelGrid plot functions to visualize initial model grid

#### Vegetation type 
- 0=Bare
- 1=Annual
- 2=Herbaceous Perennial 
- 3=Woody Perennial

In [ ]:
imshow_grid(pg, 'vegetation__plant_functional_type', at='cell',colorbar_label='Vegetation Type')
show()

In [ ]:
imshow_grid(pg, 'vegetation__live_biomass', at='cell', colorbar_label='Live biomass g')
show()

### Define model time variables
Determine basic model timestep and counter for model loop

In [ ]:
sim_end = date(sim_start_date.year+tot_sim_len_y,sim_start_date.month,sim_start_date.day)
dt = min([veg_time_step_d,env_time_step_d])
data['dt']=dt
day_freq = str(dt)+'D'
date_counter = pd.date_range(start=sim_start_date,end=sim_end,freq=day_freq).strftime('%Y-%m-%d').tolist()
step_counter=range(len(date_counter))
model_counter=tuple(zip(step_counter,date_counter))

Initialize crops PlantGrowth class with veg_params from Excel file

In [ ]:
crops=PlantGrowth(pg,**vegparams,dt) #need to update this to reflect changes in init key

## Main model loop script
This is the main model loop that controls the simulation that
- Advances model time 
- Updates environmental variables
- Simulates changes to vegetation communities
- Stores outputs

In [ ]:
for i in model_counter:
    #update temp attirbute on grid
    crops.run_one_step()
    #if this is a save timestep, save grid object to netcdf

In [ ]:
temp_dir = tempfile.mkdtemp()
os.chdir(temp_dir)
write_netcdf("test.nc", pg, format="NETCDF3_64BIT", time=step_counter[0])